# Cleaning birdstrike dataset
Because our dataset only contains the names of airports, we need to look up the coordinates a different way. Unfortunately we also don't have the ICAO or IATA codes, which would make it easy. Since the names might have a different notation we use Google Maps, which is a safe bet for accurate info.

## Initialize
First we import the necessary libraries. Then we load our birdstrike dataset and see what it looks like.

In [1]:
import requests
import json
import urllib.parse

import pandas as pd
from getpass import getpass

In [2]:
df = pd.read_csv("Bird Strikes Test.csv", low_memory=False)
df.head()

,Airport: Name,Altitude bin,Aircraft: Make/Model,Effect: Impact to flight,FlightDate,Record ID,Effect: Indicated Damage,Aircraft: Number of engines?,Aircraft: Airline/Operator,Origin State,...,When: Time (HHMM),When: Time of day,Pilot warned of birds or wildlife?,Cost: Aircraft time out of service (hours),Cost: Other (inflation adj),Cost: Repair (inflation adj),Cost: Total $,Miles from airport,Feet above ground,Speed (IAS) in knots
0,DETROIT METRO WAYNE COUNTY ARPT,< 1000 ft,A-319,NaN,1/1/2005 0:00,227413,No damage,2,NORTHWEST AIRLINES,Michigan,...,NaN,NaN,NaN,NaN,0,0,0,NaN,400,NaN
1,DANE COUNTY REGIONAL ARPT-TRUAX FLD,< 1000 ft,A-320,NaN,1/1/2005 0:00,226578,No damage,2,NORTHWEST AIRLINES,Wisconsin,...,NaN,NaN,NaN,NaN,0,0,0,0,0,NaN
2,HOGUE ARPT,< 1000 ft,PA-46 MALIBU,NaN,1/1/2005 0:00,268383,Caused damage,1,PRIVATELY OWNED,Tennessee,...,NaN,Day,NaN,NaN,0,0,0,0,0,20
3,SACRAMENTO INTL,< 1000 ft,B-737-300,None,1/1/2005 0:00,229514,Caused damage,2,SOUTHWEST AIRLINES,California,...,2140.0,Night,Y,NaN,0,"117,135","117,135",NaN,500,142
4,GUARULHOS INTL,< 1000 ft,B-767-300,NaN,1/1/2005 0:00,225661,No damage,2,UNITED AIRLINES,NaN,...,NaN,NaN,NaN,NaN,0,0,0,0,0,NaN


In [3]:
df.shape

(65610, 23)

In [4]:
# How many different airports are in the dataset?
df["Airport: Name"].nunique()

1448

## Get coordinates data from Google Maps API
To get geographic information about our airports we need to get the data from the Google Maps API.

In [5]:
# Set as global so not asked every time the func is called.
key = getpass()
url = f"https://maps.googleapis.com/maps/api/geocode/json?key={key}&address="

········


In [ ]:
reserved = " ! * ' ( ) ; : @ & = + $ , / ? % # [ ]"
name = "we&rd"
for letter in 
if name in reserved:
    

In [13]:
def get_coords(name):
    """Returns coordinates after query Google Maps API."""
    
    # Parse name in case it has special chars
    name = urllib.parse.quote_plus(str(name).encode('utf-8'))
    resp = requests.get(url + name)
    r = resp.json()
    
    # If something went wrong we want to return a 0,0 tuple.
    if resp.status_code != 200 or not r["results"]:
        return (0, 0)
    return tuple([val for val in r["results"][0]["geometry"]["location"].values()])

In [7]:
# Let's test if it works
lat, lng = get_coords("HOGUE ARPT")
print(f"{lat} {lng}")

35.6337237 -85.38951039999999


In [14]:
# Create a dictionary with unique airports with their coordinates
apt_dict = {apt: get_coords(apt) 
            for apt in df["Airport: Name"].drop_duplicates()}

# Then transform it to a list with those values
transposed = [[k, lat, long] 
              for k, (lat, long) in apt_dict.items()]

## Add coordinates to data
Now we can look up the coordinates for all airports and then add it to our dataframe.

In [15]:
# Feed the created list of airport data to a df
apts = pd.DataFrame(transposed, 
                    columns=["Airport: Name", "lat", "long"])

In [17]:
# Merge the coordinates data with the initial dataframe
df = pd.merge(df, apts, on="Airport: Name", how="left")

In [20]:
# Now check how many did not get any data.
df[df["lat"] ==  0].shape

(27, 25)

## Export data
Finally we save the data to a csv that we can import in Tableau.

In [21]:
df.to_csv("clean_birdstrike_data.csv")

In [22]:
# Also save the airport data so we can load later if we need to
apts.to_csv("airport_geo.csv")